# Bibliotecas

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imsave, imshow
from skimage.color import rgb2gray, gray2rgb, rgb2hsv
from sklearn.cluster import KMeans
import pathlib
from skimage import restoration, filters
from skimage import img_as_ubyte

from sklearn.mixture import GaussianMixture
from sklearn import metrics


## 1. Utilizando   a   base   de   imagens   “ultrassound.zip”   disponibilizada   no   SIGAA,   realize  a segmentação das imagens das seguintes formas:

### I. Aplique o algoritmo K-means utilizando os pixels como pesos; 

In [88]:
def kMeansTransform(path, path_img, n_clusters=8):
    
    img = imread(path_img)
    
    if len(img.shape) > 2:  img = rgb2gray(img)

    pixel_values = img.reshape((-1, 1))
    pixel_values = np.float32(pixel_values)
    
    kmeans = KMeans(n_clusters=n_clusters, n_init="auto",
                    init='k-means++').fit(pixel_values)
    
    Z_image = kmeans.predict(pixel_values)
    Z_image = Z_image.reshape(img.shape)
    
    auxCenter = min(kmeans.cluster_centers_)
    
    Z_image = (Z_image == list(kmeans.cluster_centers_).index(auxCenter))
    
    _zImageNorm = cv2.normalize(src=Z_image*255, dst=None, alpha=0,
                          beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    name_img = str(path_img).split('/')[-1]
        
    os.makedirs("../seg_kmeans/{}/".format(path), exist_ok=True)
    imsave('../seg_kmeans/{}/{}'.format(path, name_img), _zImageNorm)


In [ ]:
data_dir = pathlib.Path("../BID_fomatter/")
image_count = list(data_dir.glob('*/*.jpg'))

for img in image_count:
        if str(img).count("CNH_Frente"):
                kMeansTransform("CNH_Frente",img,3)
        elif str(img).count("CNH_Aberta"):
                kMeansTransform("CNH_Aberta",img,3)
        elif str(img).count("CNH_Verso"):
                kMeansTransform("CNH_Verso",img,3)
        elif str(img).count("CPF_Frente"):
                kMeansTransform("CPF_Frente",img,3)
        elif str(img).count("CPF_Verso"):
                kMeansTransform("CPF_Verso",img,3)
        elif str(img).count("RG_Aberto"):
                kMeansTransform("RG_Aberto",img,3)
        elif str(img).count("RG_Frente"):
                kMeansTransform("RG_Frente",img,3)
        elif str(img).count("RG_Verso"):
                kMeansTransform("RG_Verso",img,3)
        else:
                pass

In [7]:
def seg_otsu(path, path_img):
    
    img = imread(path_img)

    if len(img.shape) > 2:
        img = rgb2gray(img)

    threshold = filters.threshold_otsu(img)
    binarized_img = (img > threshold)*1
    
    binarized_img_norm = cv2.normalize(src=binarized_img, dst=None, alpha=0,
                                beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    name_img = str(path_img).split('/')[-1]

    os.makedirs("../seg_otsu/{}/".format(path), exist_ok=True)
    imsave('../seg_otsu/{}/{}'.format(path,
           name_img), binarized_img_norm)
    

In [8]:
def seg_niblack(path, path_img):

    img = imread(path_img)

    if len(img.shape) > 2:
        img = rgb2gray(img)

    threshold = filters.threshold_niblack(img)
    binarized_img = (img > threshold)*1

    binarized_img_norm = cv2.normalize(src=binarized_img, dst=None, alpha=0,
                                       beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    name_img = str(path_img).split('/')[-1]

    os.makedirs("../seg_niblack/{}/".format(path), exist_ok=True)
    imsave('../seg_niblack/{}/{}'.format(path,
           name_img), binarized_img_norm)


In [9]:
data_dir = pathlib.Path("../BID_fomatter/")
image_count = list(data_dir.glob('*/*.jpg'))

for img in image_count:
    if str(img).count("CNH_Frente"):
        seg_otsu("CNH_Frente", img)
        seg_niblack("CNH_Frente", img)
    elif str(img).count("CNH_Aberta"):
        seg_otsu("CNH_Aberta", img)
        seg_niblack("CNH_Aberta", img)
    elif str(img).count("CNH_Verso"):
        seg_otsu("CNH_Verso", img)
        seg_niblack("CNH_Verso", img)
    elif str(img).count("CPF_Frente"):
        seg_otsu("CPF_Frente", img)
        seg_niblack("CPF_Frente", img)
    elif str(img).count("CPF_Verso"):
        seg_otsu("CPF_Verso", img)
        seg_niblack("CPF_Verso", img)
    elif str(img).count("RG_Aberto"):
        seg_otsu("RG_Aberto", img)
        seg_niblack("RG_Aberto", img)
    elif str(img).count("RG_Frente"):
        seg_otsu("RG_Frente", img)
        seg_niblack("RG_Frente", img)
    elif str(img).count("RG_Verso"):
        seg_otsu("RG_Verso", img)
        seg_niblack("RG_Verso", img)
    else:
        pass


In [ ]:
img = imread(path + path_img)

if len(img.shape) > 2:
    img = rgb2gray(img)

pixel_values = img.reshape((-1, 1))
pixel_values = np.float32(pixel_values)

kmeans = GaussianMixture(
    n_components=8, random_state=0, init_params='kmeans').fit(pixel_values)

Z_image = kmeans.predict(pixel_values)
Z_image = Z_image.reshape(img.shape)

_zImageCopy = np.zeros_like(Z_image)
for x in kmeans.means_:
    if x >= 0.24 and x <= 0.40:
        new_img = Z_image == list(kmeans.means_).index(x)
        # new_img = filters.median(new_img)
        _zImageCopy += new_img

_zImageCopy = cv2.normalize(src=_zImageCopy, dst=None, alpha=0,
                            beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)


## K-Fold